In [1]:
# Parameters
cell_type = "SHSY5Y"

In [2]:
import pathlib

import numpy as np
import pandas as pd
import papermill as pm
import pyarrow as pa
import pyarrow.parquet as pq

In [3]:
# Define inputs
feature_file = pathlib.Path(f"../../data/{cell_type}_sc_norm.parquet")
feature_df = pd.read_parquet(feature_file)

In [4]:
len(feature_df["Metadata_Well"].unique())

154

In [5]:
# replace all " " with "_" in all values of the dataframe
feature_df = feature_df.replace(to_replace=" ", value="_", regex=True)

In [6]:
# remove uM in each row of the Metadata_inducer1_concentration column
feature_df["Metadata_inducer1_concentration"] = feature_df[
    "Metadata_inducer1_concentration"
].str.replace("µM", "")

In [7]:
feature_df["Metadata_inducer1_concentration"].unique()

array([None, '0.100', '1.000', '10.000', '5.000', '20.000', '0.010',
       '100.000', '2.500'], dtype=object)

In [8]:
# define output file path
feature_df_out_path = pathlib.Path(
    f"../../data/{cell_type}_preprocess_sc_norm_no_fs.parquet"
)

In [9]:
print(feature_df.shape)
feature_df.head()

(597902, 2927)


,Metadata_cell_type,Metadata_Well,Metadata_number_of_singlecells,Metadata_Site,Metadata_incubation inducer (h),Metadata_inhibitor,Metadata_inhibitor_concentration,Metadata_inhibitor_concentration_unit,Metadata_inducer1,Metadata_inducer1_concentration,...,Nuclei_Texture_Variance_CorrGasdermin_3_02_256,Nuclei_Texture_Variance_CorrGasdermin_3_03_256,Nuclei_Texture_Variance_CorrMito_3_00_256,Nuclei_Texture_Variance_CorrMito_3_01_256,Nuclei_Texture_Variance_CorrMito_3_02_256,Nuclei_Texture_Variance_CorrMito_3_03_256,Nuclei_Texture_Variance_CorrPM_3_00_256,Nuclei_Texture_Variance_CorrPM_3_01_256,Nuclei_Texture_Variance_CorrPM_3_02_256,Nuclei_Texture_Variance_CorrPM_3_03_256
0,SH-SY5Y,B13,3765,1,6,Media_ctr,NaN,None,media_ctr,None,...,0.025742,0.016018,0.294581,0.313403,0.340904,0.304771,0.077819,0.076181,0.088000,0.085886
1,SH-SY5Y,B13,3765,1,6,Media_ctr,NaN,None,media_ctr,None,...,-0.040428,-0.038309,1.540182,1.422456,1.459884,1.458574,0.014013,0.010482,0.008044,0.008553
2,SH-SY5Y,B13,3765,1,6,Media_ctr,NaN,None,media_ctr,None,...,-0.049183,-0.052954,0.002619,0.022670,0.014160,-0.006548,0.019567,0.035026,0.028281,0.011552
3,SH-SY5Y,B13,3765,1,6,Media_ctr,NaN,None,media_ctr,None,...,-0.059278,-0.059597,-0.075569,-0.078642,-0.071954,-0.071172,-0.031126,-0.030896,-0.028813,-0.029932
4,SH-SY5Y,B13,3765,1,6,Media_ctr,NaN,None,media_ctr,None,...,-0.055393,-0.056898,0.021789,0.020225,0.017540,0.007435,-0.059386,-0.056699,-0.060032,-0.059263


In [10]:
# removing costes features as they behave with great variance across all data
feature_df = feature_df.drop(feature_df.filter(regex="Costes").columns, axis=1)
print(feature_df.shape)
feature_df.head()

(597902, 2867)


,Metadata_cell_type,Metadata_Well,Metadata_number_of_singlecells,Metadata_Site,Metadata_incubation inducer (h),Metadata_inhibitor,Metadata_inhibitor_concentration,Metadata_inhibitor_concentration_unit,Metadata_inducer1,Metadata_inducer1_concentration,...,Nuclei_Texture_Variance_CorrGasdermin_3_02_256,Nuclei_Texture_Variance_CorrGasdermin_3_03_256,Nuclei_Texture_Variance_CorrMito_3_00_256,Nuclei_Texture_Variance_CorrMito_3_01_256,Nuclei_Texture_Variance_CorrMito_3_02_256,Nuclei_Texture_Variance_CorrMito_3_03_256,Nuclei_Texture_Variance_CorrPM_3_00_256,Nuclei_Texture_Variance_CorrPM_3_01_256,Nuclei_Texture_Variance_CorrPM_3_02_256,Nuclei_Texture_Variance_CorrPM_3_03_256
0,SH-SY5Y,B13,3765,1,6,Media_ctr,NaN,None,media_ctr,None,...,0.025742,0.016018,0.294581,0.313403,0.340904,0.304771,0.077819,0.076181,0.088000,0.085886
1,SH-SY5Y,B13,3765,1,6,Media_ctr,NaN,None,media_ctr,None,...,-0.040428,-0.038309,1.540182,1.422456,1.459884,1.458574,0.014013,0.010482,0.008044,0.008553
2,SH-SY5Y,B13,3765,1,6,Media_ctr,NaN,None,media_ctr,None,...,-0.049183,-0.052954,0.002619,0.022670,0.014160,-0.006548,0.019567,0.035026,0.028281,0.011552
3,SH-SY5Y,B13,3765,1,6,Media_ctr,NaN,None,media_ctr,None,...,-0.059278,-0.059597,-0.075569,-0.078642,-0.071954,-0.071172,-0.031126,-0.030896,-0.028813,-0.029932
4,SH-SY5Y,B13,3765,1,6,Media_ctr,NaN,None,media_ctr,None,...,-0.055393,-0.056898,0.021789,0.020225,0.017540,0.007435,-0.059386,-0.056699,-0.060032,-0.059263


In [11]:
# replacing '/' in treatment dosage column to avoid errors in file interpolation including such strings
feature_df = feature_df.replace(to_replace="/", value="_per_", regex=True)

In [12]:
# replace nan values with 0

columns_to_fill = [
    "Metadata_inducer1_concentration",
    "Metadata_inducer2_concentration",
    "Metadata_inhibitor_concentration",
]
feature_df[columns_to_fill].fillna(0, inplace=True)

/tmp/ipykernel_100325/2784960678.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_df[columns_to_fill].fillna(0, inplace=True)


In [13]:
# replace all None values with 0
feature_df["Metadata_inducer1_concentration"].fillna(0, inplace=True)

In [14]:
# create a list of columns to be converted to float
col_list = [
    "Metadata_inducer1_concentration",
    "Metadata_inducer2_concentration",
    "Metadata_inhibitor_concentration",
]
# loop through the list and convert each column to float
for i in col_list:
    feature_df[i] = feature_df[i].apply(
        lambda x: f"{float(x):.3f}" if float(x) != 0 else float(x)
    )

In [15]:
len(feature_df["Metadata_Well"].unique())

154

#### Combine Inducer1 and Inducer2 into one column

In [16]:
# treatment column merge
conditions = [
    (feature_df["Metadata_inducer2"].isnull()),
    feature_df["Metadata_inducer2"].notnull(),
]

results = [
    (feature_df["Metadata_inducer1"]).astype(str),
    (
        feature_df["Metadata_inducer1"]
        + "_"
        + feature_df["Metadata_inducer2"].astype(str)
    ),
]
feature_df["Metadata_Treatment"] = np.select(condlist=conditions, choicelist=results)


# dose column merge
results = [
    (
        feature_df["Metadata_inducer1_concentration"].astype(str)
        + "_"
        + feature_df["Metadata_inducer1_concentration_unit"].astype(str)
    ),
    (
        feature_df["Metadata_inducer1_concentration"].astype(str)
        + "_"
        + feature_df["Metadata_inducer1_concentration_unit"].astype(str)
        + "_"
        + feature_df["Metadata_inducer2_concentration"].astype(str)
        + "_"
        + feature_df["Metadata_inducer2_concentration_unit"].astype(str)
    ),
]
feature_df["Metadata_Dose"] = np.select(condlist=conditions, choicelist=results)

## N Beta Column condition generation
columns generated to used for linear modeling where terms separated by '__' will be a beta coefficient 

In [17]:
# one beta of inudcer1, inducer1 concentration, inhibitor, and inhibitor concentration all as 1 beta term
feature_df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"] = (
    feature_df["Metadata_Treatment"]
    + "_"
    + feature_df["Metadata_Dose"].astype(str)
    # + "_"
    # + feature_df['Metadata_inducer1_concentration_unit'].astype(str)
    + "_"
    + feature_df["Metadata_inhibitor"].astype(str)
    + "_"
    + feature_df["Metadata_inhibitor_concentration"].astype(str)
    + "_"
    + feature_df["Metadata_inhibitor_concentration_unit"].astype(str)
).astype(str)


# two beta of inducer1, inhibitor, and inhibitor concentration all as 1 beta term + inducer1 concentration as 2nd beta term
feature_df["twob_Metadata_Treatment_Dose_Inhibitor_Dose"] = (
    feature_df["Metadata_Treatment"]
    + "_"
    + feature_df["Metadata_inhibitor"].astype(str)
    + "_"
    + feature_df["Metadata_inhibitor_concentration"].astype(str)
    + "__"
    + feature_df["Metadata_Dose"].astype(str)
).astype(str)

# three beta of inducer 1 as 1 beta term, inducer1 concentration as 2nd beta term, inhibitor and inhibitor concentration as 3rd beta term
feature_df["threeb_Metadata_Treatment_Dose_Inhibitor_Dose"] = (
    feature_df["Metadata_Treatment"]
    + "__"
    + feature_df["Metadata_Dose"].astype(str)
    + "__"
    + feature_df["Metadata_inducer1_concentration_unit"].astype(str)
    + "_"
    + feature_df["Metadata_inhibitor"].astype(str)
    + "_"
    + feature_df["Metadata_inhibitor_concentration"].astype(str)
).astype(str)

# four beta of inducer 1 as 1 beta term, inducer1 concentration as 2nd beta term, inhibitor as 3rd beta term, and inhibitor concentration as 4th beta term
feature_df["fourb_Metadata_Treatment_Dose_Inhibitor_Dose"] = (
    feature_df["Metadata_Treatment"]
    + "__"
    + feature_df["Metadata_Dose"].astype(str)
    + "__"
    + feature_df["Metadata_inducer1_concentration_unit"].astype(str)
    + "_"
    + feature_df["Metadata_inhibitor"].astype(str)
    + "__"
    + feature_df["Metadata_inhibitor_concentration"].astype(str)
).astype(str)

In [18]:
replacement_dict = {
    "None": "0",
    "µ": "u",
    "nan": "0",
}
for pattern, replacement in replacement_dict.items():
    print(pattern, replacement)
    feature_df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"] = feature_df[
        "oneb_Metadata_Treatment_Dose_Inhibitor_Dose"
    ].replace(to_replace=str(pattern), value=str(replacement), regex=True)

None 0
µ u
nan 0


In [19]:
feature_df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"] = feature_df[
    "oneb_Metadata_Treatment_Dose_Inhibitor_Dose"
].str.replace("media_ctr_0.0_0_Media_ctr_0_0", "media_ctr_0.0_0_Media_ctr_0.0_0")

/tmp/ipykernel_100325/1031947954.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  ].str.replace("media_ctr_0.0_0_Media_ctr_0_0", "media_ctr_0.0_0_Media_ctr_0.0_0")


In [20]:
# need to convert to strings to save as parquet
# if the column is an object then convert it to a string
for column in feature_df.columns:
    if feature_df[column].dtype == "object":
        feature_df[column] = feature_df[column].astype(str)

In [21]:
print(cell_type, len(feature_df["Metadata_Well"].unique()))

SHSY5Y 154


In [23]:
# write to parquet file
feature_df.to_parquet(feature_df_out_path)